In [6]:
import rasterio
from rasterio.features import geometry_mask
import geopandas as gpd
from shapely.geometry import box
import geopandas as gpd
import fiona.crs as crs

import os
import glob

In [7]:
def clip_shapefile(sar_image_path, output_file_path, input_file_path):

    with rasterio.open(sar_image_path) as src:
        left, bottom, right, top = src.bounds

        # Specify the clipping extent in EPSG:4326 (WGS 84)
    clip_extent = gpd.GeoDataFrame(geometry=[box(left, bottom, right, top)], crs=crs.CRS.from_epsg(4326))

    input_data = gpd.read_file(input_file_path)

    # Clip the data to the specified extent
    clipped_data = gpd.overlay(input_data, clip_extent, how='intersection')

    # Save the clipped data to a new shapefile
    clipped_data.to_file(output_file_path, driver='ESRI Shapefile')


In [8]:
def create_binary_image(sar_image_path, shapefile_path, output_image_path):
    # Read SAR image
    with rasterio.open(sar_image_path) as src:
        sar_data = src.read(1)
        transform = src.transform
        profile = src.profile

    # Read shapefile
    gdf = gpd.read_file(shapefile_path)
    print(sar_data.shape)
    mask = geometry_mask(gdf.geometry, out_shape=sar_data.shape, transform=transform, invert=True)

    # Create a new image with the same dimensions as the SAR image
    binary_image = mask.astype('uint8') * 255

    # Update the profile for the output image
    profile.update(count=1, dtype='uint8')

    # Write the new image
    with rasterio.open(output_image_path, 'w', **profile) as dst:
        dst.write(binary_image, 1)

In [9]:
def generate_water_masks(input_dir):
    input_files = sorted(glob.glob(os.path.join(input_dir, '*.tif')))
    input_shapefile_path = "./Brazil_Shapefile/Areas_merged.shp"
    
    for input_file in input_files:
        filename = input_file.replace("./data\\", "")
        filename_tif = filename.replace(".tif", "")
        shapefile_path = "./shapefiles/"+filename_tif+".shp"

        clip_shapefile(input_file, shapefile_path, input_shapefile_path)

        output_file_path = "./output_tif/"+filename_tif+"_new_mask.tif"
        create_binary_image(input_file, shapefile_path, output_file_path)
        print("Done "+filename)

In [10]:
if __name__ == "__main__":
    generate_water_masks("./data/")

(3008, 4000)
Done subset_0_of_subset_1_of_final_S1A_IW_GRDH_1SDV_20240109T095534_20240109T095559_052028_06499B_56C7.tif
(3008, 4000)
Done subset_10_of_subset_1_of_final_S1A_IW_GRDH_1SDV_20240131T101122_20240131T101151_052349_065480_4193.tif
(3008, 4000)
Done subset_11_of_subset_1_of_final_S1A_IW_GRDH_1SDV_20240131T101241_20240131T101306_052349_065480_3565.tif
(3008, 4000)
Done subset_12_of_subset_1_of_final_S1A_IW_GRDH_1SDV_20240131T101241_20240131T101306_052349_065480_3565.tif
(3008, 4000)
Done subset_13_of_subset_1_of_final_S1A_IW_GRDH_1SDV_20240131T101216_20240131T101241_052349_065480_5D46.tif
(3008, 4000)
Done subset_14_of_subset_1_of_final_S1A_IW_GRDH_1SDV_20240131T101216_20240131T101241_052349_065480_5D46.tif
(3008, 4000)
Done subset_15_of_subset_2_of_final_S1A_IW_GRDH_1SDV_20240109T095534_20240109T095559_052028_06499B_56C7.tif
(3008, 4000)
Done subset_16_of_subset_2_of_final_S1A_IW_GRDH_1SDV_20240109T095534_20240109T095559_052028_06499B_56C7.tif
(3008, 4000)
Done subset_17_of_su